In [1]:
from collections import Counter
from os.path import join


input_path = "."
txt = open(join(input_path, "lenin.txt"), "r").read().lower()
cnt = Counter(txt)
charset = [i for i in cnt.keys() if cnt[i] > 1000]
to_char = dict(enumerate(charset))
to_id = dict(zip(charset, range(len(charset))))
charset = set(charset)
data = [to_id[c] for c in txt if c in charset]

# should be ~35m
len(to_id.keys()), len(data)

(92, 35243900)

In [2]:
import keras
import keras.layers as L

BATCH_LEN = 32
N_CHARS = len(to_char)
SEQ_LEN = 100
PER_EPOCH = len(data) // (BATCH_LEN * SEQ_LEN * 10)


def get_model():
    model = keras.models.Sequential()
    model.add(L.InputLayer(input_shape=(100,), batch_size=32, dtype='int32'))
    model.add(L.Embedding(N_CHARS, 100, input_length=SEQ_LEN))
    model.add(L.GRU(1024,
                    recurrent_activation='sigmoid',
                    recurrent_initializer='glorot_uniform',
                    stateful=True,
                    return_sequences=True))
    model.add(L.Dense(N_CHARS))
    model.add(L.Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = get_model()
model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (32, 100, 100)            9200      
_________________________________________________________________
gru_1 (GRU)                  (32, 100, 1024)           3456000   
_________________________________________________________________
dense_1 (Dense)              (32, 100, 92)             94300     
_________________________________________________________________
activation_1 (Activation)    (32, 100, 92)             0         
Total params: 3,559,500
Trainable params: 3,559,500
Non-trainable params: 0
_________________________________________________________________


In [3]:
from random import randint
import numpy as np
from keras.utils import to_categorical


def generate_batches():
    X_b = np.zeros((BATCH_LEN, SEQ_LEN))
    y_b = np.zeros((BATCH_LEN, SEQ_LEN, N_CHARS))
    while True:
        for i in range(BATCH_LEN):
            current_idx = randint(0, len(data))
            while current_idx + SEQ_LEN >= len(data):
                current_idx = randint(0, len(data))
            
            X_b[i, :] = data[current_idx : current_idx + SEQ_LEN]
            temp_y = data[current_idx + 1: current_idx + SEQ_LEN + 1]
            y_b[i, :, :] = to_categorical(temp_y, num_classes=N_CHARS)
        yield X_b, y_b


num_epochs = 4
history = model.fit_generator(generate_batches(),
                    PER_EPOCH,
                    num_epochs)
model.save('lenin_model.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/4
1101/1101 [==============================] - 319s 289ms/step - loss: 2.4103
Epoch 2/4
1101/1101 [==============================] - 316s 287ms/step - loss: 1.7601
Epoch 3/4
1101/1101 [==============================] - 316s 287ms/step - loss: 1.6381
Epoch 4/4
1101/1101 [==============================] - 315s 287ms/step - loss: 1.6248


In [0]:
from google.colab.files import download
download('lenin_model.h5')

In [0]:
def decode(seq):
    return "".join(to_char[round(i)] for i in seq)
  
def encode(seq):
    return [to_id[i] for i in seq]

  
def gen_txt(prefix="", power=2):
    model.reset_states()
    X_b, y_b = next(iter(generate_batches()))
    cur_vec = X_b[0]
    cur_vec[-len(prefix):] = np.array(encode(prefix))
    cur_vec[:-len(prefix)] = np.array(encode(" " * (100 - len(prefix))))
    cur_vec = np.array([cur_vec for i in range(32)])

    texts = [prefix for i in range(32)]

    for i in range(900):
        output = model.predict(cur_vec)[:,-1]
        output = (output ** power) / (output ** power).sum(axis=1).reshape(32, 1)
        for j in range(32):
            new_val = np.random.choice(range(N_CHARS), p=output[j])
            cur_vec[j] = np.concatenate((cur_vec[j][1:], [new_val]))
            texts[j] += decode(cur_vec[j][-1:])
    return texts


def print_txt(texts):
    for i in range(len(texts)):
        print("NUMBER", i + 1)
        for j, c in enumerate(texts[i]):
            print(c, end="")
            if j % 80 == 79:
                print()
        print("\n\n")

In [8]:
%%time

TXT_NUM = 5

print_txt(gen_txt(" ", 1.5)[:TXT_NUM])

NUMBER 1
 «социал-демократической революции» в россии как буржуазия о нем бывалоов оверпе
ти за отечество» (см. к. дапиток революционной программе никак как в английская,
 буржуазным рокамет политика на этой гриименной политики на  нет ничегоое стоки 
на свободных стного стоять против промышленности за власть и рабочих.
все колони
альные коми стоит в особенности опитания в большевистских странах. выстепевно вы
ше не связан современного производства – просто принтипи1.
написано в москве  пр
и известных словах народной политики вяже и беспо5я, сделано. но так надо разъяс
нения, что не отрицал пам были иного от оценки примиритозилов не только только н
астроению противоречия. опытно случает полчить до принии русской политики «искры
» и «революционное явление с обеспечить как на существно принят в об акене собир
ается сентябрь 1893 г.
декобнейшего скрыто позволите на производство они пойти н
а ве с кпссе. когда ш


NUMBER 2
 домоды в статьях на место крестьянскую фракцию и группы «в–произво

In [9]:
print_txt(gen_txt(" ", 2)[:TXT_NUM])

NUMBER 1
 наше положение в попытки рабочих и крестьенариду силы польского совета рабочих 
и солдатских депутатов. статья ленина «рабочий класс и выступал против ленина от
 27 декабря 1905 г.
впервые напечатано в 1922 г. в журнале «коммунистический отч
ет на заседании совета рабочих и солидарности с войней в первой мировой войны – 
армия не противоречит от капиталистических постановлений вооружений пролетариата
  выражали пролетариат видеть, что они не содержанию место свое  выбор под письм
ом о том, что до сих пор статья о «пролетариата» («рабочие в или при этом съезд 
во всей своей программе, следовательно, рабочих, а на печвый довод на себераторс
кой работе. кроме такого производства, построения дальнейшего строя и старались 
совершил колебания производственной политикой и классовому массом не правительст
во колебания и др. различия  на московской газетами советской работы. в 1905–190
7 годах –еорои в коми


NUMBER 2
 конституционные западноевропейских стачечников и мелких производит

In [10]:
print_txt(gen_txt(" ", 3)[:TXT_NUM])

NUMBER 1
 поддержку «народных комиссариатов» (сочинения, 4 изд., том 20, стр. 144–152.
по
становления в россии в первой мировой войны – секретарь совета рабочих и солдатс
ких депутатов советской республики от пролетариата и советской республике война 
система военной политики и против войны данного положения в россии на состав орг
анизации к нем в состоянии поддерживать на политику советской власти в оппортуни
стическом обществе о промышленном положении рабочих в пролетариат от войн против
 военной политики пролетариата на самостоятельное время о том, что в письме в ре
дакцию «пролетария» № 4, 2 июня 1905 г.
печатается по тексту газеты «пролетарий»
 № 4, стр. 279–270. подпись: в. ильин, в. и.

печатается по тексту газеты «правд
а» № 25, 25 апреля 1900 г.
печатается по тексту газеты «пролетарий» № 35, 1 (18)
 апреля 1905 г.
печатается по тексту газеты «пролетарий» № 15, 25 апреля 1914 г.

печатается по тексту


NUMBER 2
 не подтверждение только в том численном съезде советов рабочих в с

In [11]:
print_txt(gen_txt(" ", 5)[:TXT_NUM])

NUMBER 1
 противоречие о программе о программе и во всей россии и народной социал-демокра
тии, в 1905 году – профессор французской социал-демократии. в 1881 году вошел в 
советской россии в 1905 году работал в профессиональных союзников с революционно
й борьбе с полной политикой и пролетариата и пролетариата и более высоком положе
нии в профессии советской республике и на промышленности в состав революции в ро
ссии против народного социалистического и политической экономии, которое в нашем
 положении дел в россии на рабочих в советской россии в 1895 году всего народа в
 советской россии в 1895 году вошел в редакцию «пролетария» № 15, 25 апреля 190 
года в № 1 «демократической партии» (1895), «социал-демократ», [генерал германии
, в 1881–1885 годах – председатель центрального органа совета рабочих и солдатск
их депутатов и советской россии при обсуждении вопроса о работе в россии в 1891 
году всего в германии


NUMBER 2
 советской республики. в 1905 году – один из основателей войны – со

In [12]:
%%time

print_txt(gen_txt(" я призываю ", 1.75)[:TXT_NUM])

NUMBER 1
 я призываю к позицию с.-д. действительности и т. д.еаобразованные к брошюре раб
очих, поддави на основе перед нами остановиться на неприятельственной политике н
а пролетариату и писал «в пемообразелев к вопросу о врем демократии, а ко случае
и полуии пролетариатскойпроперцели в  оппортунистической программе в петр газете
 «пролетами» примера к капиталистическому врема в сторо газета «речь» № 50, 15 о
ктября 1903 г. и а с совеализмом и его попыток «вполне противоречие масс дел не 
для группы разрыва социалисизма, обещает самостоятельное отношение к ней, от про
долже себе против трудовой группы особенно решательно по крайней очень  возникот
ь, образования и устройство и тте организации и все стачечное словечка с тем она
 ее точно совершенно наверод мелким свободах, повторяюо не возможность прот ново
го употребления реакции и вильсотрудничество в россии не получить его обновить и
 желание на границей и капиталис


NUMBER 2
 я призываю крайность рабочих в состоянии производства в

In [13]:
%%time
print_txt(gen_txt(" да здравствует ", 2.25)[:TXT_NUM])

NUMBER 1
 да здравствует в массах и в 1894 году в процессе вопроса о выборах в состав раб
отал против военной россии при отсутствии противников бел выступления на самой р
езкой статье «за страница свободы крестьянства к социализмом, которое мы видим д
елать не соглашат советской организации, на этот вопрос о профессиональных союзо
в на меньшевиков и как гработе на пример обычной статьей «по суда, которое до си
х пор не может получить его советскую роль к другим устройством в советской рево
люции, которые стоит его себе все только постановления о реальных городах в перв
ой русской революции в процент политики и советской революции, развития этой ста
тье «объединительной работе» («материали настоящий помещиков и крестьянски настр
оение масс одной из отработоченной рабочей партии, о состав цк и в 1906 году и с
татьи по вопросу о работах поддержку закона о работе социал-демократической печа
ти. – 242.
спектор книг пг. рода вой


NUMBER 2
 да здравствует распространить этот попытки старых п